In [35]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import os

# Настройки
pio.renderers.default = "browser"
np.random.seed(42)
os.makedirs("results", exist_ok=True)

# Генерация фигур
def manual_center(pcd):
    pcd = copy.deepcopy(pcd)
    pcd.translate(-pcd.get_center())
    return pcd

def place_on_top(obj_to_move, obj_base, offset=0.0):
    z_base_max = obj_base.get_max_bound()[2]
    z_move_min = obj_to_move.get_min_bound()[2]
    obj_to_move.translate((0, 0, z_base_max - z_move_min + offset))
    return obj_to_move

def generate_assets():
    # Снеговик
    s1 = o3d.geometry.TriangleMesh.create_sphere(1.0).sample_points_uniformly(1500)
    s2 = o3d.geometry.TriangleMesh.create_sphere(0.7).sample_points_uniformly(1000).translate((0, 0, 1.2))
    s3 = o3d.geometry.TriangleMesh.create_sphere(0.4).sample_points_uniformly(800).translate((0, 0, 2.0))
    nose = o3d.geometry.TriangleMesh.create_cone(0.1, 0.4).sample_points_uniformly(200)
    nose.rotate(nose.get_rotation_matrix_from_xyz((-np.pi / 2, 0, 0))).translate((0, -0.4, 2.0))
    snowman = manual_center(s1 + s2 + s3 + nose)
    
    # Стол
    table_w, table_h, table_t, leg_h = 3.0, 2.0, 0.2, 1.5
    top = manual_center(o3d.geometry.TriangleMesh.create_box(table_w, table_h, table_t).sample_points_uniformly(2500)).translate((0, 0, leg_h + table_t / 2))
    leg = manual_center(o3d.geometry.TriangleMesh.create_cylinder(0.1, leg_h).sample_points_uniformly(400)).translate((0, 0, leg_h / 2))
    dx, dy = table_w / 2 - 0.2, table_h / 2 - 0.2
    table = top
    for x in [dx, -dx]:
        for y in [dy, -dy]:
            table += copy.deepcopy(leg).translate((x, y, 0))
    
    # Пирамида
    cube = manual_center(o3d.geometry.TriangleMesh.create_box(1, 1, 1).sample_points_uniformly(400))
    pyramid = o3d.geometry.PointCloud()
    for z, size in enumerate([3, 2, 1]):
        for x in range(size):
            for y in range(size):
                pyramid += copy.deepcopy(cube).translate((x - (size - 1) / 2, y - (size - 1) / 2, z + 0.5))

    # Звезда
    beam = manual_center(o3d.geometry.TriangleMesh.create_box(3, 0.4, 0.4).sample_points_uniformly(400)).translate((1.5, 0, 0))
    star_geometries = [copy.deepcopy(beam).rotate(beam.get_rotation_matrix_from_xyz((0, 0, np.deg2rad(i * 60)))) for i in range(6)]
    star = o3d.geometry.PointCloud()
    for geom in star_geometries: star += geom
    star = manual_center(star)
    
    return snowman, table, pyramid, star

def assemble_scene():
    snowman, table, pyramid, star = generate_assets()
    
    snowman_s = copy.deepcopy(snowman); place_on_top(snowman_s, table)
    star_under = copy.deepcopy(star); star_under.scale(0.6, center=star_under.get_center()); star_under.translate((0.0, 0.0, -star_under.get_min_bound()[2] + 0.1))
    pyramid_s = copy.deepcopy(pyramid); pyramid_s.translate((4.0, 0, 0))
    star_above = copy.deepcopy(star); star_above.scale(0.8, center=star_above.get_center()); star_above.rotate(star_above.get_rotation_matrix_from_xyz((0, np.pi / 2, 0)), center=star_above.get_center()); star_above.translate((4.0, 0.0, pyramid_s.get_max_bound()[2] - star_above.get_min_bound()[2] + 0.6))
    full_scene_base = table + snowman_s + star_under + pyramid_s + star_above

    pts = np.asarray(full_scene_base.points)
    # Добавляем шум 1.5 см
    full_scene_base.points = o3d.utility.Vector3dVector(pts + np.random.normal(0, 0.015, pts.shape))
    full_scene_base.paint_uniform_color([0.6, 0.6, 0.6])
    
    return full_scene_base

# Визуализация
def plot_objects(objects, title, filename_suffix, parameters_text=""):
    data = []
    for obj in objects:
        if isinstance(obj, o3d.geometry.PointCloud):
            pts = np.asarray(obj.points)
            cols = np.asarray(obj.colors)
            if len(cols) == 0: cols = 'gray'
            if len(pts) > 40000:
                idx = np.random.choice(len(pts), 40000, replace=False)
                pts = pts[idx]
                if not isinstance(cols, str): cols = cols[idx]
            data.append(go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers',
                                     marker=dict(size=2, color=cols), name='Points', showlegend=False))
        elif isinstance(obj, o3d.geometry.TriangleMesh):
             # Solid Hull (Mesh3d)
            vertices = np.asarray(obj.vertices)
            triangles = np.asarray(obj.triangles)
            if obj.has_vertex_colors():
                c = np.asarray(obj.vertex_colors)[0]
                rgb = f'rgb({int(c[0]*255)}, {int(c[1]*255)}, {int(c[2]*255)})'
            else: rgb = 'red'
            
            data.append(go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2],
                                  i=triangles[:,0], j=triangles[:,1], k=triangles[:,2],
                                  color=rgb, opacity=0.4, name='Hull'))

    layout = go.Layout(title=f"<b>{title}</b><br><sup>{parameters_text}</sup>", 
                       scene=dict(aspectmode='data', xaxis_visible=False, yaxis_visible=False, zaxis_visible=False))
    fig = go.Figure(data=data, layout=layout)
    fig.write_html(f"results/{filename_suffix}.html")
    fig.show()

# DBSCAN
def run_dbscan_full(pcd):
    # Labels
    print("DBSCAN...")
    labels = np.array(pcd.cluster_dbscan(eps=0.35, min_points=20, print_progress=False))
    max_label = labels.max()
    
    cols = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    cols[labels < 0] = 0
    pcd_cl = copy.deepcopy(pcd)
    pcd_cl.colors = o3d.utility.Vector3dVector(cols[:, :3])
    
    # Визуализация: DBSCAN Clustering
    plot_objects([pcd_cl], "DBSCAN Clustering", "02_dbscan_clusters", 
                 f"eps=0.35, min_points=20. Found {max_label+1} clusters")
    
    # Hulls
    print("Convex Hulls...")
    pcd_gray = copy.deepcopy(pcd)
    pcd_gray.paint_uniform_color([0.6, 0.6, 0.6])
    geoms = [pcd_gray]
    
    if 0 <= max_label < 50:
        for i in range(max_label + 1):
            idx = np.where(labels == i)[0]
            if len(idx) > 10:
                try:
                    hull, _ = pcd.select_by_index(idx).compute_convex_hull()
                    if hull.is_watertight():
                        hull.paint_uniform_color(plt.get_cmap("tab20")(i/max_label)[:3])
                        geoms.append(hull)
                except: pass
                
    # Визуализация: Hulls
    plot_objects(geoms, "DBSCAN + Convex Hulls", "03_dbscan_hulls", 
                 "Convex hulls wrapped around detected density clusters")

if __name__ == "__main__":
    scene = assemble_scene()
    plot_objects([scene], "Scene", "01_raw")
    run_dbscan_full(scene)

DBSCAN...
Convex Hulls...


In [36]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import os

pio.renderers.default = "browser"
os.makedirs("results", exist_ok=True)
np.random.seed(42)

# Пирамида

def get_noisy_pyramid():
    cube = o3d.geometry.TriangleMesh.create_box(1, 1, 1).sample_points_uniformly(1500) 
    cube.translate(-cube.get_center())
    pcd = o3d.geometry.PointCloud()
    for z, size in enumerate([3, 2, 1]):
        for x in range(size):
            for y in range(size):
                pcd += copy.deepcopy(cube).translate((x - (size - 1) / 2, y - (size - 1) / 2, z + 0.5))
    
    pts = np.asarray(pcd.points)
    # Шум оставляем (он нужен для проверки алгоритма), но на патчах мы его скроем
    pcd.points = o3d.utility.Vector3dVector(pts + np.random.normal(0, 0.015, pts.shape))
    pcd.paint_uniform_color([0.6, 0.6, 0.6])
    return pcd


# Визуализация

def plot_result(objects, title, filename):
    data = []
    for obj in objects:
        # RANSAC (Points)
        if isinstance(obj, o3d.geometry.PointCloud):
            pts = np.asarray(obj.points); cols = np.asarray(obj.colors)
            if len(cols)==0: cols='gray'
            data.append(go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers',
                                     marker=dict(size=5, color=cols), name='Plane', showlegend=False))
            
        # PATCHES (Solid Boxes) 
        elif isinstance(obj, o3d.geometry.OrientedBoundingBox):
            corners = np.asarray(obj.get_box_points())
            i=[7,0,0,0,4,4,6,6,4,0,3,2]; j=[3,4,1,2,5,6,5,2,0,1,6,3]; k=[0,7,2,3,6,7,1,1,5,5,7,6]
            c = obj.color if hasattr(obj,'color') else [0,1,0]
            rgb = f'rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
            # opacity=1.0, чтобы непрозрачные, гладкие
            data.append(go.Mesh3d(x=corners[:,0], y=corners[:,1], z=corners[:,2],
                                  i=i, j=j, k=k, color=rgb, opacity=1.0, flatshading=True, name='Patch'))
            
    fig = go.Figure(data=data, layout=go.Layout(title=title, scene=dict(aspectmode='data')))
    fig.write_html(f"results/{filename}.html")
    fig.show()

def run_pyramid_ransac(pcd):
    print("RANSAC...")
    rest = copy.deepcopy(pcd)
    geoms = []
    
    # Ищем грани
    for i in range(15):
        if len(rest.points) < 50: break
        
        plane_model, inliers = rest.segment_plane(distance_threshold=0.04, ransac_n=3, num_iterations=1000)
        
        if len(inliers) < 50: break
            
        plane = rest.select_by_index(inliers)
        plane.paint_uniform_color(plt.get_cmap("tab20")(i)[:3])
        geoms.append(plane)
        rest = rest.select_by_index(inliers, invert=True)
    
    # Остатки (шум) здесь не рисуем, чтобы картинка была чище, только найденные плоскости
    
    plot_result(geoms, "PYRAMID: RANSAC", "pyramid_01_ransac")

def run_pyramid_patches(pcd):
    print("Patches...")
    
    # Параметры 
    pcd_norm = copy.deepcopy(pcd)
    pcd_norm.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd_norm.orient_normals_consistent_tangent_plane(50)
    
    oboxes = pcd_norm.detect_planar_patches(
        normal_variance_threshold_deg=30,
        coplanarity_deg=30,
        outlier_ratio=0.75,
        min_plane_edge_length=0.15,
        min_num_points=10, 
        search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)
    )
    
    for ob in oboxes: ob.color = np.random.rand(3)
    
    plot_result(oboxes, "PYRAMID: Planar Patches", "pyramid_02_patches")

if __name__ == "__main__":
    pyramid = get_noisy_pyramid()
    run_pyramid_ransac(pyramid)
    run_pyramid_patches(pyramid)

RANSAC...
Patches...


In [37]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import os

pio.renderers.default = "browser"
os.makedirs("results", exist_ok=True)
np.random.seed(42)

# Стол

def get_noisy_table():
    table_w, table_h, table_t, leg_h = 3.0, 2.0, 0.2, 1.5
    
    def manual_center(pcd):
        pcd.translate(-pcd.get_center())
        return pcd

    # Меши
    mesh_top = o3d.geometry.TriangleMesh.create_box(table_w, table_h, table_t)
    mesh_top = manual_center(mesh_top).translate((0, 0, leg_h + table_t / 2))
    
    mesh_leg = o3d.geometry.TriangleMesh.create_cylinder(0.1, leg_h)
    mesh_leg = manual_center(mesh_leg).translate((0, 0, leg_h / 2))
    
    dx, dy = table_w / 2 - 0.2, table_h / 2 - 0.2
    
    full_mesh = mesh_top + \
                copy.deepcopy(mesh_leg).translate((dx, dy, 0)) + \
                copy.deepcopy(mesh_leg).translate((-dx, dy, 0)) + \
                copy.deepcopy(mesh_leg).translate((dx, -dy, 0)) + \
                copy.deepcopy(mesh_leg).translate((-dx, -dy, 0))
    
    # Плотное облако
    pcd = full_mesh.sample_points_uniformly(number_of_points=10000) 
    
    pts = np.asarray(pcd.points)
    pcd.points = o3d.utility.Vector3dVector(pts + np.random.normal(0, 0.015, pts.shape))
    pcd.paint_uniform_color([0.6, 0.6, 0.6])
    
    return pcd


# Визуализация
def plot_result(objects, title, filename):
    data = []
    for obj in objects:
        # Фон для Patches
        if isinstance(obj, o3d.geometry.PointCloud):
            pts = np.asarray(obj.points); cols = np.asarray(obj.colors)
            if len(cols)==0: cols='gray'

            is_background = (np.mean(cols) < 0.6) 
            
            size = 3 if is_background else 4
            opacity = 0.6 if is_background else 1.0 
            
            data.append(go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers',
                                     marker=dict(size=size, color=cols, opacity=opacity), 
                                     name='Points', showlegend=False))
            
        # PATCHES
        elif isinstance(obj, o3d.geometry.OrientedBoundingBox):
            corners = np.asarray(obj.get_box_points())
            i=[7,0,0,0,4,4,6,6,4,0,3,2]; j=[3,4,1,2,5,6,5,2,0,1,6,3]; k=[0,7,2,3,6,7,1,1,5,5,7,6]
            c = obj.color if hasattr(obj,'color') else [0,1,0]
            rgb = f'rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
            data.append(go.Mesh3d(x=corners[:,0], y=corners[:,1], z=corners[:,2],
                                  i=i, j=j, k=k, color=rgb, opacity=1.0, flatshading=True, name='Patch'))
            
    fig = go.Figure(data=data, layout=go.Layout(title=title, scene=dict(aspectmode='data')))
    fig.write_html(f"results/{filename}.html")
    fig.show()

# Анализ
def run_table_ransac(pcd):
    print("RANSAC...")
    rest = copy.deepcopy(pcd)
    geoms = []
    
    # Ищем плоскости
    for i in range(6):
        if len(rest.points) < 100: break
        plane_model, inliers = rest.segment_plane(distance_threshold=0.04, ransac_n=3, num_iterations=1000)
        if len(inliers) < 100: break
            
        plane = rest.select_by_index(inliers)
        plane.paint_uniform_color(plt.get_cmap("tab10")(i)[:3])
        geoms.append(plane)
        rest = rest.select_by_index(inliers, invert=True)
    
    # Добавляем ножки облаками, чтобы их было видно
    rest.paint_uniform_color([0.4, 0.4, 0.4]) 
    geoms.append(rest)
    
    plot_result(geoms, "TABLE: RANSAC (Planes)", "table_01_ransac")

def run_table_patches(pcd):
    print("Patches...")
    
    pcd_norm = copy.deepcopy(pcd)
    pcd_norm.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd_norm.orient_normals_consistent_tangent_plane(50)
    
    oboxes = pcd_norm.detect_planar_patches(
        normal_variance_threshold_deg=30, 
        coplanarity_deg=30,               
        outlier_ratio=0.75,
        min_plane_edge_length=0.2,        
        min_num_points=50, 
        search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)
    )
    
    for ob in oboxes: ob.color = np.random.rand(3)
    
    pcd_background = copy.deepcopy(pcd)
    pcd_background.paint_uniform_color([0.4, 0.4, 0.4]) 
    
    plot_result([pcd_background] + oboxes, "TABLE: Patches + Points (Legs)", "table_02_patches")

if __name__ == "__main__":
    table = get_noisy_table()
    run_table_ransac(table)
    run_table_patches(table)

RANSAC...
Patches...


In [38]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import os

pio.renderers.default = "browser"
os.makedirs("results", exist_ok=True)
np.random.seed(42)

# Звезда
def get_noisy_star():
    def manual_center(geometry):
        geometry.translate(-geometry.get_center())
        return geometry

    beam_mesh = o3d.geometry.TriangleMesh.create_box(3, 0.5, 0.5)
    beam_mesh = manual_center(beam_mesh).translate((1.5, 0, 0))
    pcd = o3d.geometry.PointCloud()
    
    for i in range(6):
        R = beam_mesh.get_rotation_matrix_from_xyz((0, 0, np.deg2rad(i * 60)))
        beam_rot = copy.deepcopy(beam_mesh).rotate(R, center=(0,0,0))
        pcd += beam_rot.sample_points_uniformly(3000)

    pts = np.asarray(pcd.points)
    # Шум 0.015
    pcd.points = o3d.utility.Vector3dVector(pts + np.random.normal(0, 0.015, pts.shape))
    pcd.paint_uniform_color([0.6, 0.6, 0.6])
    return pcd

# Визуализация
def create_plotly_mesh(o3d_mesh, color_rgb):
    vertices = np.asarray(o3d_mesh.vertices)
    triangles = np.asarray(o3d_mesh.triangles)
    if len(vertices) == 0 or len(triangles) == 0: return None
    rgb_str = f'rgb({int(color_rgb[0]*255)},{int(color_rgb[1]*255)},{int(color_rgb[2]*255)})'
    
    return go.Mesh3d(
        x=vertices[:,0], y=vertices[:,1], z=vertices[:,2],
        i=triangles[:,0], j=triangles[:,1], k=triangles[:,2],
        color=rgb_str, opacity=1.0, flatshading=False, name='Solid Surface'
    )

def plot_result(objects, title, filename):
    data = []
    for obj in objects:
        if isinstance(obj, o3d.geometry.PointCloud):
            pts = np.asarray(obj.points); cols = np.asarray(obj.colors)
            if len(cols)==0: cols='gray'
            data.append(go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers',
                                     marker=dict(size=4, color=cols), name='Pts', showlegend=False))
        elif isinstance(obj, go.Mesh3d):
            data.append(obj)
            
    fig = go.Figure(data=data, layout=go.Layout(title=title, scene=dict(aspectmode='data')))
    fig.write_html(f"results/{filename}.html")
    fig.show()


# Анализ
def run_star_ransac(pcd):
    print("RANSAC...")
    rest = copy.deepcopy(pcd)
    geoms = []
    
    for i in range(14):
        if len(rest.points) < 400: break
        
        plane_model, inliers = rest.segment_plane(distance_threshold=0.08, ransac_n=3, num_iterations=1000)
        
        # Min Points 400, чтобы боковые грани не пропадали, но мусор отсеивался
        if len(inliers) < 400: break
            
        plane = rest.select_by_index(inliers)
        plane.paint_uniform_color(plt.get_cmap("tab20")(i)[:3])
        geoms.append(plane)
        rest = rest.select_by_index(inliers, invert=True)
    
    plot_result(geoms, "STAR: RANSAC", "star_01_ransac_bal")


def run_star_alpha_shapes(pcd):
    print("Alpha Shapes...") # Иначе патчи всю звезду захватывают 
    rest = copy.deepcopy(pcd)
    plotly_objects = []
    
    for i in range(14):
        if len(rest.points) < 400: break
        
        plane_model, inliers = rest.segment_plane(distance_threshold=0.08, ransac_n=3, num_iterations=1000)
        
        if len(inliers) < 400: break
            
        plane_pcd = rest.select_by_index(inliers)
        color = plt.get_cmap("tab20")(i)[:3]
        
        try:
            mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(plane_pcd, alpha=0.25)
            mesh.compute_vertex_normals()
            plotly_mesh = create_plotly_mesh(mesh, color)
            if plotly_mesh:
                plotly_objects.append(plotly_mesh)
        except Exception: pass

        rest = rest.select_by_index(inliers, invert=True)
    
    plot_result(plotly_objects, "STAR: Alpha Shapes", "star_02_alpha_bal")

if __name__ == "__main__":
    star = get_noisy_star()
    run_star_ransac(star)
    run_star_alpha_shapes(star)

RANSAC...
Alpha Shapes...


In [39]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import os

pio.renderers.default = "browser"
os.makedirs("results", exist_ok=True)
np.random.seed(42)

# Снеговик
def get_noisy_snowman():
    def manual_center(geometry):
        geometry.translate(-geometry.get_center())
        return geometry

    s1 = o3d.geometry.TriangleMesh.create_sphere(1.0).sample_points_uniformly(3000)
    s2 = o3d.geometry.TriangleMesh.create_sphere(0.7).sample_points_uniformly(2000).translate((0, 0, 1.2))
    s3 = o3d.geometry.TriangleMesh.create_sphere(0.4).sample_points_uniformly(1000).translate((0, 0, 2.0))
    nose = o3d.geometry.TriangleMesh.create_cone(0.1, 0.4).sample_points_uniformly(300)
    nose.rotate(nose.get_rotation_matrix_from_xyz((-np.pi / 2, 0, 0))).translate((0, -0.4, 2.0))
    
    snowman = manual_center(s1 + s2 + s3 + nose)
    pts = np.asarray(snowman.points)
    snowman.points = o3d.utility.Vector3dVector(pts + np.random.normal(0, 0.015, pts.shape))
    snowman.paint_uniform_color([0.6, 0.6, 0.6])
    return snowman

# Визуализация
def plot_result(objects, title, filename):
    data = []
    for obj in objects:
        if isinstance(obj, o3d.geometry.PointCloud):
            pts = np.asarray(obj.points); cols = np.asarray(obj.colors)
            if len(cols)==0: cols='gray'
            # Фон
            is_bg = (np.mean(cols) < 0.7)
            size = 3 if is_bg else 5
            opacity = 0.5 if is_bg else 1.0
            
            data.append(go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers',
                                     marker=dict(size=size, color=cols, opacity=opacity), name='Pts', showlegend=False))
            
        elif isinstance(obj, o3d.geometry.OrientedBoundingBox):
            corners = np.asarray(obj.get_box_points())
            i=[7,0,0,0,4,4,6,6,4,0,3,2]; j=[3,4,1,2,5,6,5,2,0,1,6,3]; k=[0,7,2,3,6,7,1,1,5,5,7,6]
            c = obj.color if hasattr(obj,'color') else [0,1,0]
            rgb = f'rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
            data.append(go.Mesh3d(x=corners[:,0], y=corners[:,1], z=corners[:,2],
                                  i=i, j=j, k=k, color=rgb, opacity=1.0, flatshading=True, name='Patch'))
            
    fig = go.Figure(data=data, layout=go.Layout(title=title, scene=dict(aspectmode='data')))
    fig.write_html(f"results/{filename}.html")
    fig.show()


# Анализ
def run_snowman_ransac(pcd):
    print("RANSAC...")
    rest = copy.deepcopy(pcd)
    geoms = []
    
    for i in range(3):
        if len(rest.points) < 50: break
        plane_model, inliers = rest.segment_plane(distance_threshold=0.04, ransac_n=3, num_iterations=1000)
        if len(inliers) < 100: break 
            
        plane = rest.select_by_index(inliers)
        plane.paint_uniform_color(plt.get_cmap("tab10")(i)[:3])
        geoms.append(plane)
        rest = rest.select_by_index(inliers, invert=True)
    
    rest.paint_uniform_color([0.5, 0.5, 0.5])
    geoms.append(rest)
    
    plot_result(geoms, "SNOWMAN: RANSAC", "snowman_01_ransac")

def run_snowman_patches(pcd):
    print("Patches...")
    
    pcd_norm = copy.deepcopy(pcd)
    pcd_norm.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd_norm.orient_normals_consistent_tangent_plane(50)
    
    oboxes = pcd_norm.detect_planar_patches(
        normal_variance_threshold_deg=5,  
        coplanarity_deg=5,               
        outlier_ratio=0.05,             
        min_plane_edge_length=0.3,       
        min_num_points=30, 
        search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)
    )
    
    print(f"Patches found: {len(oboxes)}") # Должно быть 0
    
    for ob in oboxes: ob.color = np.random.rand(3)
    
    # Рисуем только серый фон
    pcd_bg = copy.deepcopy(pcd)
    pcd_bg.paint_uniform_color([0.5, 0.5, 0.5])
    
    plot_result([pcd_bg] + oboxes, "SNOWMAN: Patches", "snowman_02_patches")

if __name__ == "__main__":
    man = get_noisy_snowman()
    run_snowman_ransac(man)
    run_snowman_patches(man)

RANSAC...
Patches...
Patches found: 0


In [41]:
import open3d as o3d
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import os

pio.renderers.default = "browser"
os.makedirs("results", exist_ok=True)
np.random.seed(42)

# Конусная звезда
def get_noisy_cone_star():
    def manual_center(geometry):
        geometry.translate(-geometry.get_center())
        return geometry

    cone_mesh = o3d.geometry.TriangleMesh.create_cone(radius=0.5, height=3.0)
    
    R_lay = cone_mesh.get_rotation_matrix_from_xyz((0, np.pi/2, 0))
    cone_mesh.rotate(R_lay, center=(0,0,0))
    
    pcd = o3d.geometry.PointCloud()
    
    for i in range(6):
        R = cone_mesh.get_rotation_matrix_from_xyz((0, 0, np.deg2rad(i * 60)))
        cone_rot = copy.deepcopy(cone_mesh).rotate(R, center=(0,0,0))
        pcd += cone_rot.sample_points_uniformly(2500)

    pts = np.asarray(pcd.points)
    # Шум 0.015
    pcd.points = o3d.utility.Vector3dVector(pts + np.random.normal(0, 0.015, pts.shape))
    pcd.paint_uniform_color([0.6, 0.6, 0.6])
    
    return pcd

# Визуализация
def plot_result(objects, title, filename):
    data = []
    for obj in objects:
        if isinstance(obj, o3d.geometry.PointCloud):
            pts = np.asarray(obj.points); cols = np.asarray(obj.colors)
            if len(cols)==0: cols='gray'
            
            is_bg = (np.mean(cols) < 0.7)
            size = 3 if is_bg else 5
            opacity = 0.5 if is_bg else 1.0
            
            data.append(go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers',
                                     marker=dict(size=size, color=cols, opacity=opacity), name='Pts', showlegend=False))
            
        elif isinstance(obj, o3d.geometry.OrientedBoundingBox):
            corners = np.asarray(obj.get_box_points())
            i=[7,0,0,0,4,4,6,6,4,0,3,2]; j=[3,4,1,2,5,6,5,2,0,1,6,3]; k=[0,7,2,3,6,7,1,1,5,5,7,6]
            c = obj.color if hasattr(obj,'color') else [0,1,0]
            rgb = f'rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})'
            data.append(go.Mesh3d(x=corners[:,0], y=corners[:,1], z=corners[:,2],
                                  i=i, j=j, k=k, color=rgb, opacity=1.0, flatshading=True, name='Patch'))
            
    fig = go.Figure(data=data, layout=go.Layout(title=title, scene=dict(aspectmode='data')))
    fig.write_html(f"results/{filename}.html")
    fig.show()


# Анализ
def run_cone_ransac(pcd):
    print("RANSAC...")
    rest = copy.deepcopy(pcd)
    geoms = []
    
    # Ищем полоски вдоль конусов
    for i in range(12):
        if len(rest.points) < 50: break
        
        plane_model, inliers = rest.segment_plane(distance_threshold=0.04, ransac_n=3, num_iterations=1000)
        
        if len(inliers) < 100: break
            
        plane = rest.select_by_index(inliers)
        plane.paint_uniform_color(plt.get_cmap("tab20")(i)[:3])
        geoms.append(plane)
        rest = rest.select_by_index(inliers, invert=True)
    
    # Фон
    rest.paint_uniform_color([0.5, 0.5, 0.5])
    geoms.append(rest)
    
    plot_result(geoms, "CONE STAR: RANSAC", "cone_01_ransac")

def run_cone_patches(pcd):
    print("Patches...")
    
    pcd_norm = copy.deepcopy(pcd)
    pcd_norm.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd_norm.orient_normals_consistent_tangent_plane(50)
    
    oboxes = pcd_norm.detect_planar_patches(
        normal_variance_threshold_deg=5, 
        coplanarity_deg=5,               
        outlier_ratio=0.1,                
        min_plane_edge_length=0.3,
        min_num_points=30, 
        search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)
    )
    
    print(f"Patches found: {len(oboxes)} (Expect 0)")
    
    for ob in oboxes: ob.color = np.random.rand(3)
    
    pcd_bg = copy.deepcopy(pcd)
    pcd_bg.paint_uniform_color([0.5, 0.5, 0.5])
    
    plot_result([pcd_bg] + oboxes, "CONE STAR: Patches", "cone_02_patches")

if __name__ == "__main__":
    cone_star = get_noisy_cone_star()
    run_cone_ransac(cone_star)
    run_cone_patches(cone_star)

RANSAC...
Patches...
Patches found: 0 (Expect 0)
